In [1]:
from ClusterTree import *
from ReadWritePickleFile import *
from graphviz import Digraph, nohtml

In [2]:
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):
        self.items.insert(0,item)

    def dequeue(self):
        return self.items.pop()

    def size(self):
        return len(self.items)


def TreeVisualization(tree, filename):
    node = tree.getRoot()
    g = Digraph('g', filename=filename, node_attr={'shape': 'record', 'height': '.1'})
    queue = Queue()
    i = 0
    str1 = '''A:{}, AT:{}, V:{}, OC:{}'''.format(node.getAttribute(), node.getAttributeType(), node.getDecidingValue(),
                                           node.getOperationCode())
    queue.enqueue(('node'+str(i), node))
    nodeVal = '<f0> |<f1> {}|<f2>'.format(str1)
    g.node('node'+str(i), nohtml(nodeVal))
    i+=1
    while not queue.isEmpty(): 
  
            # Dequeue a vertex from  
            # queue and print it 
            keyValue = queue.dequeue()
            key = keyValue[0]
            val = keyValue[1]
            if not val.isLeaf():
                lChild = val.getLeft()
                if not lChild.isLeaf():
                    str1 ='''A:{}, AT:{}, 
                             V:{}, OC:{}
                             '''.format(lChild.getAttribute(), lChild.getAttributeType()[0],
                                                    lChild.getDecidingValue(), lChild.getOperationCode())
                else:
                    str1 = '''CID: {}
                           '''.format(lChild.getClusterId()[lChild.getClusterId().find("__")+2:])
                    
                queue.enqueue(('node'+str(i), lChild))
                nodeVal = '<f0> |<f1> {}|<f2>'.format(str1)
                g.node('node'+str(i), nohtml(nodeVal))
                g.edge('{}:f0'.format(key), '{}:f1'.format('node'+str(i)))
                i+=1
                if val.getRight():
                    rChild = val.getRight()
                    if not rChild.isLeaf():
                        str2 ='''A:{}, AT:{}, 
                             V:{}, OC:{}
                             '''.format(rChild.getAttribute(), rChild.getAttributeType()[0],
                                                    rChild.getDecidingValue(), rChild.getOperationCode())
                    else:
                        str2 = '''CID: {}
                           '''.format(rChild.getClusterId()[rChild.getClusterId().find("__")+2:])
                        
                    queue.enqueue(('node'+str(i), rChild))
                    nodeVal = '<f0> |<f1> {}|<f2>'.format(str2)
                    g.node('node'+str(i), nohtml(nodeVal))
                    g.edge('{}:f2'.format(key), '{}:f1'.format('node'+str(i)))
                    i+=1
                
    g.view() 

In [3]:
tree = readPicklefile('dataAUHISB_06_11_2019_mixed_356815161718_v1_200_pt001.pickle')

In [4]:
filename = './Tree.gv'
TreeVisualization(tree, filename)

In [5]:
def DEBUG_getClusterID(info, tree):
        ## Iterative
        assert isinstance(info, dict), logger.error("ClusterTree.getClusterID() expects information to be passed in\
        python dictionary(key-value pair) format!")

        info = dict((k.lower().strip().lower(), v) for k, v in info.items())

        curNode = tree.root
        clusterID = None
        while clusterID is None and curNode is not None:
            if curNode.isLeaf():
                clusterID = curNode.getClusterId()
                break

            val = curNode.getDecidingValue()
            opcode = curNode.getOperationCode()
            att = curNode.getAttribute()
            acceptedDtypes = tree.attributeToDataType[att]

            if str in acceptedDtypes:
                if not isinstance(val, str):
                    val = str(val)

                val = str(val).strip().lower()

            nodeVal = info.get(att, tree.defaultVals[att])
            if isinstance(nodeVal, str):
                nodeVal = nodeVal.strip().lower()
                if isinstance(nodeVal, str) and \
                        ((nodeVal[0] == '-' and nodeVal.replace('-','',1).replace('.','',1).isdecimal()) or
                         (nodeVal[0] != '-' and nodeVal.replace('.','',1).isdecimal())):
                    nodeVal = str(float(nodeVal))
            else:
                nodeVal = str(float(nodeVal))
            
            acceptedDtypes = tree.attributeToDataType[att]
            flag = False
            for typ in acceptedDtypes:
                flag = flag or isinstance(nodeVal, typ)

            if not flag:
                if isinstance(nodeVal, str):
                    nodeVal = float(nodeVal)
                    
            print ("Deciding Value: {}, Attribute: {}, Opcode: {}, Current Vlaue: {}".format(val, att, opcode, nodeVal))
            if opcode == 1:
                if isinstance(nodeVal, str):
                    logger.error("Unexpected data for Attribute: {}, Passed Value: {}, Numerical data expected!".format(att, nodeVal))
                    nodeVal = self.defaultVals[att]
                if nodeVal<= val:
                    curNode = curNode.getLeft()
                    print ("Going Left")
                else:
                    curNode = curNode.getRight()
                    print ("Going Right")
            elif opcode == 2:
                if nodeVal not in val:
                    
                    curNode = curNode.getLeft()
                    print ("Going Left")
                else:
                    curNode = curNode.getRight()
                    print ("Going Right")
            elif opcode == 3:
                if nodeVal != val:
                    curNode = curNode.getLeft()
                    print ("Going Left")
                else:
                    curNode = curNode.getRight()
                    print ("Going Right")

        return clusterID, 2

In [6]:
tree = readPicklefile('Integration-Test-10-withOption2-mergingCentroids_V5_1_Silhouette.pickle')

In [7]:
filename = './TreeTest10.gv'
TreeVisualization(tree, filename)

In [10]:
info = {'arrivaldate-weekday': 'Tuesday', 'leaddays': 7, 'channel-id': 1}

In [12]:
DEBUG_getClusterID(info, tree)

Deciding Value: ['3.0'], Attribute: channel-id, Opcode: 2, Current Vlaue: 1.0
Going Left
Deciding Value: 3.0, Attribute: lang-id, Opcode: 3, Current Vlaue: 1.0
Going Left
Deciding Value: 12.0, Attribute: channel-id, Opcode: 3, Current Vlaue: 1.0
Going Left
Deciding Value: 157.0, Attribute: leaddays, Opcode: 1, Current Vlaue: 7.0
Going Left
Deciding Value: 1.0, Attribute: channel-id, Opcode: 3, Current Vlaue: 1.0
Going Right


('test-w/option-2-mergingcentroids_v5_1_silhouette__cluster_default', 2)